In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import silhouette_samples, silhouette_score, v_measure_score
from sklearn.datasets import load_files
import re
from unidecode import unidecode
from mpl_toolkits import mplot3d
from yellowbrick.cluster import KElbowVisualizer
print("Bibliotecas Baixadas")


nltk.download('rslp')
nltk.download('stopwords')


Bibliotecas Baixadas


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Carregue os DataFrames origem_ID e text do CSV
caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )
text['ID'] = range(1, len(text) + 1)


In [3]:
#Diminuir o dataframe

num_linhas_text = len(text)
print("Número de linhas no DataFrame:", num_linhas_text)

#origem_ID = origem_ID.drop(range(250000), axis=0)
origem_ID = origem_ID.drop_duplicates(subset=["classif"], keep="first")
num_linhas_classif = len(origem_ID)
print("Número de linhas no DataFrame:", num_linhas_classif)

Número de linhas no DataFrame: 978
Número de linhas no DataFrame: 19351


In [5]:
valor_a_apagar = " ARMAMENTO"
origem_ID=origem_ID[origem_ID["grupo"] != valor_a_apagar]

valor_a_apagar1 = " MATERIAIS BELICOS NUCLEARES"
origem_ID=origem_ID[origem_ID["grupo"] != valor_a_apagar1]

valor_a_apagar2 = " EQUIPAMENTOS DE TIRO"
origem_ID=origem_ID[origem_ID["grupo"] != valor_a_apagar2]

valor_a_apagar3 = " MISSEIS GUIADOS"
origem_ID=origem_ID[origem_ID["grupo"] != valor_a_apagar3]

valor_a_apagar4 = " EQUIPAMENTOS PARA LANCAMENTOS, POUSO E MANOBRA DE AERONAVES"
origem_ID=origem_ID[origem_ID["grupo"] != valor_a_apagar4]



num_linhas= len(origem_ID)
print("Número de linhas no DataFrame:", num_linhas)

Número de linhas no DataFrame: 19175


In [6]:
#Limpando text.txt
text['dsc_item'] = text['dsc_item'].astype(str)
origem_ID['classif'] = origem_ID['classif'].astype(str)
text["dsc_item"] = text["dsc_item"].apply(lambda x: x.lower()) #converte todas as letras para minúsculo
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: x.lower()) #converte todas as letras para minúsculo 
text["dsc_item"] = text["dsc_item"].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
text["dsc_item"] = text["dsc_item"].apply(lambda x: unidecode(x)) #remove acentos
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: unidecode(x)) #remove acentos

stemmer = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
text["dsc_item"] = text["dsc_item"].apply(lambda x: stemmer.stem(x))

stemmer1 = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: stemmer.stem(x))


In [7]:
# Crie um DataFrame vazio 'resultado'
resultado = pd.DataFrame(columns=['ID', 'dsc_item', 'melhor_classif', 'match_score'])

# Itere sobre as linhas do DataFrame 'text'
for index, row in text.iterrows():
    dsc_item_text = row['dsc_item']
    ID = row['ID']
    melhor_classif = None
    melhor_score = 0

    # Itere sobre as linhas do DataFrame 'origem_ID' para encontrar o melhor match fuzzy
    for _, row_origem in origem_ID.iterrows():
        classif = row_origem['classif']
        score = fuzz.ratio(dsc_item_text, classif)  # Use o método 'ratio' para o score de match fuzzy

        if score > melhor_score:
            melhor_score = score
            melhor_classif = classif

    # Adicione a linha correspondente ao DataFrame 'resultado'
    resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)

# O DataFrame 'resultado' agora contém os resultados do matching fuzzy
print(resultado)
#Alterar Depois--------------------------------------------------------
    # Crie um DataFrame temporário para esta linha
    #temp_df = pd.DataFrame({'ID': [ID], 'dsc_item': [dsc_item_text], 'melhor_classif': [melhor_classif], 'match_score': [melhor_score]})

    # Concatene o DataFrame temporário ao 'resultado'
    #resultado = pd.concat([resultado, temp_df], ignore_index=True)

# O DataFrame 'resultado' agora contém os resultados do matching fuzzy
#print(resultado)

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

...

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)
C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_cl

      ID                                           dsc_item  \
0      1                                         agua sanit   
1      2                                  biscoito agua e s   
2      3                                 agua mineral c20lt   
3      4  sabao em po  cor azul composto por tensoativo ...   
4      5                                agua destilada 10ml   
..   ...                                                ...   
973  974                 uniao pvc para solda 85 mm agua fr   
974  975  uniao pvc sold pagua fria predial 20mmun 00009905   
975  976  valvula de retencao em bronze de 3 polegada ho...   
976  977  vasilhame para agua mineral galao com capacida...   
977  978                          vasilhame de agua 20 litr   

                                        melhor_classif match_score  
0                                           agua sanit          95  
1                                     botao para estof          59  
2                                   

C:\Users\caios\AppData\Local\Temp\ipykernel_9092\1676076088.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultado = resultado.append({'ID': ID, 'dsc_item': dsc_item_text, 'melhor_classif': melhor_classif, 'match_score': melhor_score}, ignore_index=True)


In [8]:
nome_arquivo_saida = 'resultado_23_10.csv'
# Use o método to_csv para salvar o DataFrame com o nome especificado
resultado.to_csv(nome_arquivo_saida, index=False) 

resultado.head(5)



,ID,dsc_item,melhor_classif,match_score
0,1,agua sanit,agua sanit,95
1,2,biscoito agua e s,botao para estof,59
2,3,agua mineral c20lt,agua mineral nat,80
3,4,sabao em po cor azul composto por tensoativo ...,servicos para a infraestrutura de tecnologi...,40
4,5,agua destilada 10ml,agua destil,71


In [ ]:
valor_procurado = ' agua miner'

# Use o método loc para localizar a linha
linha_localizada = origem_ID.loc[origem_ID['classif'] == valor_procurado]

# Imprima a linha localizada
print(linha_localizada)